# Graph matrix subspace checking

In [1]:
#import os
#os.environ["MKL_NUM_THREADS"] = "4" 
#os.environ["NUMEXPR_NUM_THREADS"] = "4" 
#os.environ["OMP_NUM_THREADS"] = "4" 

### 1. First results

In [1]:
from tqdm import tqdm 
import numpy as np
import scipy as sp
import os
import datetime
from pprint import pprint

from graph_utils import *

In [2]:
def get_rank_and_nullspace(A, get_cutoff_dif=False):
    
    _, v, d = sp.linalg.svd(A, lapack_driver="gesdd")
        
    # такой выбор толеранса используется в матлабе и в нампае
    # https://numpy.org/doc/stable/reference/generated/numpy.linalg.matrix_rank.html
    tol = v.max() * max(A.shape) * np.finfo(A.dtype).eps
    nnz = (v >= tol).sum()
    
    if get_cutoff_dif == False:
        return nnz, d[nnz:].T

In [3]:
def a_is_subspace_of_b(basis_a, basis_b, get_cutoff=False):
    
    orig_rank, _ = get_rank_and_nullspace(basis_b)
    combined_rank, _ = get_rank_and_nullspace(np.hstack([basis_a, basis_b]))
    
    return orig_rank == combined_rank

In [4]:
def find_chain_starts(basis_set):
    
    # Sort by the dimensionality of subspaces
    basis_set = sorted(basis_set, key=lambda x: x[1])
    chain_starts = [basis_set[0]]
    
    # Go over basis sets and check whether their spaces are subspaces of some of chain_starts spaces
    for i in range(1, len(basis_set)):
        
        not_a_subspace = True
        
        for start in chain_starts:
            if a_is_subspace_of_b(basis_set[i][2], start[2]):
                not_a_subspace = False
                break
        
        if not_a_subspace:
            chain_starts.append(basis_set[i])
    
    return chain_starts

In [5]:
# Function for an online version of an algorithm
# chain_starts are supposed to be sorted by x[1]

def get_new_starts(chain_starts, new_basis):
    
    if len(chain_starts) == 0:
        chain_starts.append(new_basis)
        return
    
    checked_pos = 0
    
    # check if we are embedded in some space
    while checked_pos < len(chain_starts) and len(chain_starts[checked_pos][2][0]) >= len(new_basis[2][0]):
        if a_is_subspace_of_b(new_basis[2], chain_starts[checked_pos][2]):
            return
        checked_pos += 1
    
    # new basis is not embedded anywhere therefore we insert it
    chain_starts.insert(checked_pos, new_basis)
    checked_pos += 1
    
    # now we delete subspaces, that are embedded into a new one
    while checked_pos < len(chain_starts):
        if a_is_subspace_of_b(chain_starts[checked_pos][2], new_basis[2]):
            chain_starts.pop(checked_pos)
        else:
            checked_pos += 1

In [26]:
ALL_GRAPHS = get_graphs()
del ALL_GRAPHS['cycle']

# ALL_GRAPHS = {x: ALL_GRAPHS[x] for x in ['center', 'sinking_ship']}
# что-то в center баг какой-то 'sinking_ship'

In [ ]:
%%time

NON_EQUIV = True

for name, g in ALL_GRAPHS.items():
    
    print(name)
    I = find_stable_sets(g)
    A, var_matr, num_params = build_triple_equalities(I, g.shape[0])
    
    num_combs = total_num_of_combinations(I, var_matr, num_params, only_nonequivalent=NON_EQUIV)
    
    chain_starts = []
    
    for i, (addA, addB, opening) in tqdm(enumerate(build_variance_equalities_iterator(
            I, var_matr, num_params, only_nonequivalent=NON_EQUIV)), total=num_combs):
        rank, null_basis = get_rank_and_nullspace(np.array(A + addA, dtype=np.float64))
        if len(null_basis) != 0:
            get_new_starts(chain_starts, [i, rank, null_basis, opening])
    
    print("{} x {}".format(len(A), len(A[0])), "{} x {}".format(len(addA), len(addA[0])))
    print(len(chain_starts), "dimensions:", ",".join([str(len(x[2][0])) for x in chain_starts]))
    
    print("openings:")
    for x in chain_starts: 
        pprint(x[-1])

### 2. Speed optimizations

In [7]:
ALL_GRAPHS = {'dupl': ALL_GRAPHS['dupl'], 'zigzag': ALL_GRAPHS['zigzag'], 'fork': ALL_GRAPHS['fork'], 
              'big_triag': ALL_GRAPHS['big_triag'], 'big_zig': ALL_GRAPHS['big_zig']}

In [8]:
# storm has multiple bases
# ship stil doesn't cut it.. we'll have to try more cores

# if you simply switch lapack driver to gesvd - results don't change but it takes twice longer

In [ ]:
%%time

for name, g in ALL_GRAPHS.items():

    print(name)
    I = find_stable_sets(g)
    A, var_matr, num_params = build_triple_equalities(I, g.shape[0])

    basis_set = []
    for i, addA in enumerate(build_variance_equalities_iterator(I, var_matr, num_params, only_nonequivalent=True)):
        basis_set.append([i, *get_rank_and_nullspace(np.array(A + addA[0], dtype=np.float64))])

    chain_starts = find_chain_starts(basis_set)

    print("{} x {}".format(len(A), len(A[0])), "{} x {}".format(len(addA[0]), len(addA[0][0])))
    print(len(chain_starts), "dimensions:", ",".join([str(len(x[2][0])) for x in chain_starts]))

In [9]:
%%time
# it has been 2min29sec
# with 8 cores it's 2min58sec
# with 4 cores it's 3min3sec

for name, g in ALL_GRAPHS.items():
    
    print(name)
    I = find_stable_sets(g)
    A, var_matr, num_params = build_triple_equalities(I, g.shape[0])
    
    chain_starts = []
    for i, addA in enumerate(build_variance_equalities_iterator(I, var_matr, num_params, only_nonequivalent=True)):
        chain_starts.append([i, *get_rank_and_nullspace(np.array(A + addA[0], dtype=np.float64))])
        chain_starts = find_chain_starts(chain_starts)
    
    print("{} x {}".format(len(A), len(A[0])), "{} x {}".format(len(addA[0]), len(addA[0][0])))
    print(len(chain_starts), "dimensions:", ",".join([str(len(x[2][0])) for x in chain_starts]))

storm
19 x 45 35 x 45
2 dimensions: 7,7
CPU times: user 12min 5s, sys: 9.58 s, total: 12min 15s
Wall time: 3min 3s


In [12]:
%%time

maxdif = 1e-19

for name, g in ALL_GRAPHS.items():
    
    print(name)
    I = find_stable_sets(g)
    A, var_matr, num_params = build_triple_equalities(I, g.shape[0])
    
    chain_starts = []
    for i, addA in enumerate(build_variance_equalities_iterator(I, var_matr, num_params, only_nonequivalent=True)):
        
        matrix = np.array(A + addA[0], dtype=np.float64)
        _, v, d = sp.linalg.svd(matrix, lapack_driver="gesdd")
        tol = v.max() * max(matrix.shape) * np.finfo(matrix.dtype).eps
        nnz = (v >= tol).sum()
        
        if nnz < len(v):
            maxdif = max(v[nnz] / v[nnz-1], maxdif)
            
        get_new_starts(chain_starts, [i, nnz, d[nnz:].T])
    
    print("{} x {}".format(len(A), len(A[0])), "{} x {}".format(len(addA[0]), len(addA[0][0])))
    print(len(chain_starts), "dimensions:", ",".join([str(len(x[2][0])) for x in chain_starts]))

storm
19 x 45 35 x 45
2 dimensions: 7,7
CPU times: user 17min 15s, sys: 16.3 s, total: 17min 32s
Wall time: 2min 11s


#### no aplas optimizations

mkl is used, 4 cores

online (stupid): 1min 32s
offline: 1min 18s
online (smarter): 1min 25s 

#### limitting number of cores

#os.environ["MKL_NUM_THREADS"] = "1" 
#os.environ["NUMEXPR_NUM_THREADS"] = "1" 
#os.environ["OMP_NUM_THREADS"] = "1" 

online (stupid): 1min 33s
offline: 1min 20s
online (smarter): 1min 26s

#### mkl on 8 cores - same result

**so online version isn't much slower and number of cores doesn't affect the results much**

### 3. Storm analysis

In [27]:
g = ALL_GRAPHS['storm']
I = find_stable_sets(g)
A, var_matr, num_params = build_triple_equalities(I, g.shape[0])

А он правильно задан?

In [28]:
is_copirr(g)

True

Судя по обсуждениям в интернете - могут быть проблемы при применении gesdd на плохо обсуловленных матрицах - поэтому все эксперименты будем пробовать и с gesdd и gesvd 

Смотрим на максимальное соотношение между первым отрезанным и неотрезанным с.ч. и минимальное между неотрезанными с.ч. в определении подпространств 

In [60]:
max_cut_dif  = 1e-19
max_real_dif = 1e-19
min_rank = 1e3

total_num = total_num_of_combinations(I, var_matr, num_params, only_nonequivalent=True)

for addA, addB, opening in tqdm(
    build_variance_equalities_iterator(I, var_matr, num_params, only_nonequivalent=True), total=total_num
):

    matrix = np.array(A + addA, dtype=np.float64)
    _, v, d = sp.linalg.svd(matrix, lapack_driver="gesdd")
    tol = v.max() * max(matrix.shape) * np.finfo(matrix.dtype).eps
    nnz = (v >= tol).sum()
        
    if nnz < len(v):
        max_cut_dif = max(v[nnz] / v[nnz-1], max_cut_dif)
        max_real_dif = max(v[0] / v[nnz-1], max_real_dif)
        min_rank = min(nnz, min_rank)
        
print(max_cut_dif, max_real_dif)

100%|██████████| 131072/131072 [01:27<00:00, 1493.97it/s]

1.619397193511745e-14 36.66232538729725


In [62]:
len(A[0])-min_rank

7

In [59]:
max_cut_dif  = 1e-19
max_real_dif = 1e-19

total_num = total_num_of_combinations(I, var_matr, num_params, only_nonequivalent=True)

for addA, addB, opening in tqdm(
    build_variance_equalities_iterator(I, var_matr, num_params, only_nonequivalent=True), total=total_num
):

    matrix = np.array(A + addA, dtype=np.float64)
    _, v, d = sp.linalg.svd(matrix, lapack_driver="gesvd")
    tol = v.max() * max(matrix.shape) * np.finfo(matrix.dtype).eps
    nnz = (v >= tol).sum()
        
    if nnz < len(v):
        max_cut_dif = max(v[nnz] / v[nnz-1], max_cut_dif)
        max_real_dif = max(v[0] / v[nnz-1], max_real_dif)
            
print(max_cut_dif, max_real_dif)


100%|██████████| 131072/131072 [04:23<00:00, 497.56it/s]

5.682405378060707e-15 36.662325387297344


Проверим, найдётся ли среди отфильтрованных нами пространств 8-мерное

In [49]:
min_rank = 1e3

total_num = total_num_of_combinations(I, var_matr, num_params, only_nonequivalent=False)

for addA, addB, opening in tqdm(
    build_variance_equalities_iterator(I, var_matr, num_params, only_nonequivalent=False), total=total_num
):

    matrix = np.array(A + addA, dtype=np.float64)
    v = sp.linalg.svd(matrix, lapack_driver="gesdd", compute_uv=False)
    tol = v.max() * max(matrix.shape) * np.finfo(matrix.dtype).eps
    nnz = (v >= tol).sum()
    min_rank = min(nnz, min_rank)

  0%|          | 83446/34359738368 [00:35<4046:52:39, 2358.45it/s]

KeyboardInterrupt: 

Ну, этого нам не проверить